## 관측지점정보(ASOS) 전처리

In [ ]:
import pandas as pd
import glob
import os

# 폴더(관측지점정보(ASOS)) 내 csv파일 가져오기
folder_path = "C:/Users/hbi/Downloads/기상청_강수량(0523)/관측지점정보(ASOS)" # 폴더 경로
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파일명을 키로, DataFrame을 값으로 하는 딕셔너리 생성
dfs = {}
for file in csv_files:
    file_name = os.path.basename(file).replace(".csv", "")  # 확장자 제거
    dfs[file_name] = pd.read_csv(file, encoding='euc-kr')

# META_ASOS_GB 데이터프레임 확인
dfs["META_ASOS_GB"].head()

In [ ]:
# "종료일"이 NULL인 행만 남기기
for name, df in dfs.items():
    if "종료일" in df.columns:
        dfs[name] = df[df["종료일"].isnull()]

In [ ]:
# 지점, 지명, 지점주소, 위도, 경도 만 남기기
columns_to_keep = ['지점', '지점명', '지점주소', '위도', '경도']

dfs = {key: df[columns_to_keep] for key, df in dfs.items()}

In [ ]:
# 데이터프레임 출력
for name, df in dfs.items():
    print(f"\n===== {name} =====")
    print(df)

In [ ]:
# 파일로 저장
for key, df in dfs.items():
    filename = f"{key}_clean.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')

## 관측지점정보(AWS) 전처리

In [ ]:
import pandas as pd
import glob
import os

# 폴더(관측지점정보(AWS)) 내 csv파일 가져오기
folder_path = "C:/Users/hbi/Downloads/기상청_강수량(0523)/관측지점정보(AWS)" # 폴더 경
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# 파일명을 키로, DataFrame을 값으로 하는 딕셔너리 생성
dfs = {}
for file in csv_files:
    file_name = os.path.basename(file).replace(".csv", "")  # 확장자 제거
    dfs[file_name] = pd.read_csv(file, encoding='euc-kr')

# META_AWS_GB데이터프레임 확인
dfs["META_AWS_GB"].head()

In [ ]:
# "종료일"이 NULL인 행만 남기기
for name, df in dfs.items():
    if "종료일" in df.columns:
        dfs[name] = df[df["종료일"].isnull()]

In [ ]:
# 지점, 지명, 지점주소, 위도, 경도 만 남기기
columns_to_keep = ['지점', '지점명', '지점주소', '위도', '경도']

dfs = {key: df[columns_to_keep] for key, df in dfs.items()}

In [ ]:
# 데이터프레임 출력
for name, df in dfs.items():
    print(f"\n===== {name} =====")
    print(df)

In [ ]:
# 파일로 저장
for key, df in dfs.items():
    filename = f"{key}_clean.csv"
    df.to_csv(filename, index=False, encoding='utf-8-sig')

In [ ]:
# 종관기상관측(ASOS) 파일 불러기기
import os
import pandas as pd

folder_path = 'C:/Users/hbi/Downloads/기상청_강수량(0523)/종관기상관측(ASOS)'  # 폴더 경로 (같은 폴더면 '.' 으로 설정)

dfs_asos = {}  # 결과 저장할 딕셔너리

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, encoding='cp949')  # 또는 encoding='utf-8'
        key = os.path.splitext(filename)[0]  # 확장자 제거한 파일명
        dfs_asos[key] = df


In [ ]:
# 관측지점(ASOS) 파일 불러기기
import os
import pandas as pd

folder_path = 'C:/Users/hbi/Downloads/기상청_강수량(0523)/관측지점정보(ASOS)'  # 폴더 경로 (같은 폴더면 '.' 으로 설정)

dfs_asos_meta = {}  # 결과 저장할 딕셔너리

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path, encoding='utf-8-sig')  # 또는 encoding='utf-8'
        key = os.path.splitext(filename)[0]  # 확장자 제거한 파일명
        dfs_asos_meta[key] = df

In [ ]:
# ASOS파일에 관측정보지점(ASOS) '지점'으로 아우터조인 기기
key_pairs = [
    ('CN_ASOS_2022', 'META_ASOS_CN_clean'),
    ('GB_ASOS_2022', 'META_ASOS_GB_clean'),
    ('GG_ASOS_2022', 'META_ASOS_GG_clean'),
    ('CN_ASOS_2021', 'META_ASOS_CN_clean'),
    ('GB_ASOS_2021', 'META_ASOS_GB_clean'),
    ('GG_ASOS_2021', 'META_ASOS_GG_clean'),
    ('GW_ASOS_2022', 'META_ASOS_GW_clean'),
    ('GW_ASOS_2021', 'META_ASOS_GW_clean'),
    # 원하는 키 쌍 추가
]

dfs_joined = {}

for k1, k2 in key_pairs:
    df1 = dfs_asos[k1]
    df2 = dfs_asos_meta[k2]
    merged_df = df1.merge(df2, on='지점', how='left')
    dfs_joined[f'{k1}_{k2}'] = merged_df

In [ ]:
columns_order = [
    '지점', '지점명_x', '지점주소', '위도', '경도', '일시',
    '강수 계속시간(hr)', '10분 최다 강수량(mm)', '10분 최다강수량 시각(hhmi)',
    '1시간 최다강수량(mm)', '1시간 최다 강수량 시각(hhmi)', '일강수량(mm)'
]

for key, df in dfs_joined.items():
    # 존재하는 컬럼만 선택
    selected_columns = [col for col in columns_order if col in df.columns]
    df_selected = df[selected_columns].copy()
    
    # '지점명_x' -> '지점명'으로 컬럼명 변경
    if '지점명_x' in df_selected.columns:
        df_selected.rename(columns={'지점명_x': '지점명'}, inplace=True)
    
    # CSV 저장
    filename = f"{key}_clean.csv"
    df_selected.to_csv(filename, index=False, encoding='utf-8-sig')
